In [1]:
import requests
from requests import Request
from pydantic import BaseModel,Field,field_validator

class Site(BaseModel):
    站點:str = Field(alias='sitename')
    縣市:str = Field(alias='county')
    aqi:int = Field(alias='aqi')
    pm25:float = Field(alias='pm2.5_avg')
    時間:str = Field(alias='datacreationdate')

    @field_validator('pm25',mode='before')  #pm25有空字串
    @classmethod
    def space_zero(cls,value):    #space_zero名稱自訂義
        #print(type(value))
        if value == '':
            return 0.0
        return value

class AQI(BaseModel):
    records:list[Site]

aqi_url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate desc&format=JSON'
response:Request = requests.get(aqi_url)
if response.status_code == 200:      # status_code == 200 意思是資料出現200就是成功
    print('下載成功')
else:
    print('下載失敗')

data_json:str  = response.text
aqi = AQI.model_validate_json(data_json)
source_data = aqi.model_dump()['records']


下載成功


In [2]:
import pandas as pd
df1 = pd.DataFrame(source_data)
df1

,站點,縣市,aqi,pm25,時間
0,屏東(枋山),屏東縣,31,9.7,2024-03-24 15:00
1,新北(樹林),新北市,63,20.3,2024-03-24 15:00
2,屏東(琉球),屏東縣,44,13.7,2024-03-24 15:00
3,臺南(麻豆),臺南市,45,13.8,2024-03-24 15:00
4,高雄(湖內),高雄市,30,7.2,2024-03-24 15:00
...,...,...,...,...,...
995,萬華,臺北市,69,22.7,2024-03-24 04:00
996,中山,臺北市,65,21.0,2024-03-24 04:00
997,士林,臺北市,60,19.3,2024-03-24 04:00
998,淡水,新北市,53,16.2,2024-03-24 04:00


In [6]:
#如何選取多個欄位
df2 = df1[['縣市','pm25','aqi']].copy()
df2

,縣市,pm25,aqi
0,屏東縣,9.7,31
1,新北市,20.3,63
2,屏東縣,13.7,44
3,臺南市,13.8,45
4,高雄市,7.2,30
...,...,...,...
995,臺北市,22.7,69
996,臺北市,21.0,65
997,臺北市,19.3,60
998,新北市,16.2,53


In [7]:
#選取列,透過索引(index)
#如果列是數字用 iloc[]  
#如果列是文字用 loc[]   
df2.iloc[0]

縣市      屏東縣
pm25    9.7
aqi      31
Name: 0, dtype: object

In [8]:
#指定多筆索引
df2.iloc[[0,996,999]]

,縣市,pm25,aqi
0,屏東縣,9.7,31
996,臺北市,21.0,65
999,新北市,15.7,51


In [11]:
#使用iloc[:] -> 0-9筆的資料
df2.iloc[:10]

,縣市,pm25,aqi
0,屏東縣,9.7,31
1,新北市,20.3,63
2,屏東縣,13.7,44
3,臺南市,13.8,45
4,高雄市,7.2,30
5,彰化縣,18.6,59
6,彰化縣,17.2,55
7,新北市,16.3,53
8,雲林縣,17.0,55
9,臺東縣,9.1,30


In [16]:
#後10筆資料
df3 = df2.iloc[-10:]
df3

,縣市,pm25,aqi
990,桃園市,15.4,50
991,桃園市,22.4,68
992,臺北市,24.5,73
993,臺北市,23.6,71
994,臺北市,21.5,66
995,臺北市,22.7,69
996,臺北市,21.0,65
997,臺北市,19.3,60
998,新北市,16.2,53
999,新北市,15.7,51


In [15]:
df3.aqi >= 70

990    False
991    False
992     True
993     True
994    False
995    False
996    False
997    False
998    False
999    False
Name: aqi, dtype: bool

In [17]:
#搜尋資料,使用subscript
df3[df3.aqi >= 70]

,縣市,pm25,aqi
992,臺北市,24.5,73
993,臺北市,23.6,71
